# Study Workflow Automation #

## Project Setup ##

These cells involve the setup of a Study and automate steps in the workflow that can occur prior to subject enrollment.

**Pre-requisites**

1. git installed
2. Python >= 3.12

**Install dependencies: ipywdigets**

In [1]:
pip install ipywidgets mercury --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# include imports
import os
import urllib
from urllib.parse import urlparse
from ipywidgets import FileUpload
import mercury as mr
import subprocess

**Define functions to be used throughout study setup workflow**

In [3]:
# Define functions
def change_working_dir(working_dir):
    pwd = os.getcwd()

    if pwd == working_dir:
        print("INFO: Already in the {} project directory...".format(working_dir))
    else:
        print("INFO: Changing to the {} working directory...".format(working_dir))
        try:
            os.chdir(working_dir)
        except FileNotFoundError:
            print("ERROR: {} does not exist...".format(working_dir))

def git_clone(repo):
    project = urlparse(repo)[2]
    project_dir = project[project.rindex('/')+1: project.rindex('.')]
    print(project_dir)
    if project_dir:
        print("INFO: {} already exists.  Skipping clone...".format(project_dir))
    else:
        print("INFO: Cloning repository {}...".format(project_dir))
        !git clone repo
    return project_dir

## Upload and Validate the Study Design USDM JSON file ##

In [4]:
# Change to the directory into which to clone repo
project_dir = '/Users/dmoreland/projects/jupyter'
change_working_dir(project_dir)

print(os.getcwd())

# Clone repo
repo = 'https://github.com/pendingintent/cdisc-json-validation.git'
#working_dir = git_clone('https://github.com/pendingintent/cdisc-json-validation.git')
!git clone {repo}
project = urlparse(repo)[2]
working_dir = project[project.rindex('/')+1: project.rindex('.')]
print(working_dir)
change_working_dir(os.path.join(project_dir, working_dir))
print(os.getcwd())

INFO: Already in the /Users/dmoreland/projects/jupyter project directory...
/Users/dmoreland/projects/jupyter
Cloning into 'cdisc-json-validation'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 85 (delta 39), reused 61 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 1.26 MiB | 5.28 MiB/s, done.
Resolving deltas: 100% (39/39), done.
cdisc-json-validation
INFO: Changing to the /Users/dmoreland/projects/jupyter/cdisc-json-validation working directory...
/Users/dmoreland/projects/jupyter/cdisc-json-validation


In [5]:
# Install project dependencies
!pip install -r requirements.txt --quiet

In [6]:
# Create a data directory, if required
data_dir = os.path.join(project_dir,'data')
print(data_dir)
if os.path.exists(data_dir):
    print("INFO: {} exists...".format(data_dir))
else:
    print("INFO: Creating directory {}...".format(data_dir))
    os.mkdir(data_dir)

os.getcwd()

/Users/dmoreland/projects/jupyter/data
INFO: /Users/dmoreland/projects/jupyter/data exists...


'/Users/dmoreland/projects/jupyter/cdisc-json-validation'

In [7]:
# Verify correct current working directory
print(os.getcwd())

/Users/dmoreland/projects/jupyter/cdisc-json-validation


In [8]:
# Upload json file
# TODO: Replace mercury with ipywidgets FileUpload for simpler implmentation with less dependencies

file = mr.File(label="File upload", max_file_size="10MB")


mercury.File

In [10]:
# Verify the file upload adn write to local data/ directory

print(f"Uploaded file name: {file.filename}")
print(f"Uploaded file path: {file.filepath}")
dest_file = os.path.join(data_dir, file.filename)
print(dest_file)
if file.filepath is not None:
    with open(file.filepath, 'r') as file_in:
        content = file_in.read()
    with open(dest_file, 'w') as file_out:
        file_out.write(content)

Uploaded file name: pilot_LLZT_protocol.json
Uploaded file path: /var/folders/7m/j9llsw7d35s7k1_d8jc19_g80000gn/T/tmp_7g51j7w/pilot_LLZT_protocol.json
/Users/dmoreland/projects/jupyter/data/pilot_LLZT_protocol.json


In [11]:
# Execute the validation script
#import subprocess

change_working_dir(os.path.join(project_dir, working_dir))
result = subprocess.run(["python", "validate.py", dest_file], capture_output=True, text=True)


if "Validation successful" not in result.stdout:
    print("ERROR: Validation of {} not successful...".format(dest_file))
else:
    print("INFO: Validation of {} is successful...".format(dest_file))
#print('stdout:', result.stdout)
#print('stderr:', result.stderr)

INFO: Already in the /Users/dmoreland/projects/jupyter/cdisc-json-validation project directory...
INFO: Validation of /Users/dmoreland/projects/jupyter/data/pilot_LLZT_protocol.json is successful...


**If validation of the USDM JSON file is successful, the file can be used in following downstream programs**

In [12]:
change_working_dir(project_dir)

INFO: Changing to the /Users/dmoreland/projects/jupyter working directory...
